In [12]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from math import sqrt
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
import xgboost as xgb
from xgboost import XGBRegressor
from xgboost import DMatrix

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv
/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt


In [2]:
dirname='/kaggle/input/house-prices-advanced-regression-techniques'

train = pd.read_csv(os.path.join(dirname,'train.csv'))
test = pd.read_csv(os.path.join(dirname,'test.csv'))
print("TRAIN : " ,train.shape);
print("TEST : " ,test.shape);
submission = pd.read_csv(os.path.join(dirname,'sample_submission.csv'))

X = train.drop(['SalePrice'], axis=1)
y = np.log1p(train['SalePrice'])


TRAIN :  (1460, 81)
TEST :  (1459, 80)


In [3]:
# Get list of categorical variables
s = (X.dtypes == 'object')
object_cols = list(s[s].index)

print("Categorical variables:")
print(object_cols)

catcol = (X.dtypes == 'object')
categorical_cols = list(catcol[catcol].index)
numcol = (X.dtypes != 'object')
numerical_cols = list(numcol[numcol].index)


Categorical variables:
['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']


In [4]:
# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

In [5]:
#model = RandomForestRegressor(n_estimators=500, random_state=0)
model = XGBRegressor(n_estimators=250, max_depth=3,learning_rate=0.055,num_boost_round=267,subsample=0.5,colsample_bytree=0.5,objective='reg:squarederror',n_jobs=4,silet=False)

In [6]:
# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

In [7]:
## Handling missing values

# Make copy to avoid changing original data (when imputing)
X_plus = X.copy()

# Make new columns indicating what will be imputed
# Get names of columns with missing values
cols_with_missing = [col for col in X.columns
                     if X[col].isnull().any()]
for col in cols_with_missing:
    X_plus[col + '_was_missing'] = X_plus[col].isnull()

In [8]:
"""
# Multiply by -1 since sklearn calculates *negative* MAE
scores =  np.sqrt(-1 * cross_val_score(my_pipeline, X_plus, y,
                              cv=5,
                              scoring='neg_mean_squared_error'))
print("Average Cross Validation RMSE score (across experiments):")
print(scores.mean())
"""

'\n# Multiply by -1 since sklearn calculates *negative* MAE\nscores =  np.sqrt(-1 * cross_val_score(my_pipeline, X_plus, y,\n                              cv=5,\n                              scoring=\'neg_mean_squared_error\'))\nprint("Average Cross Validation RMSE score (across experiments):")\nprint(scores.mean())\n'

In [9]:
preprocessor.fit(X_plus)
X_cv = preprocessor.transform(X_plus)

DX_plus = DMatrix(data=X_cv, label=y.to_numpy())
params = {
		'n_estimators': 250,
		'learning_rate': 0.055,
		'subsample':0.5,
		'colsample_bytree':0.5,
		'objective':'reg:squarederror', 
		'max_depth':3
	}
# Perform cross-validation with early stopping: cv_results
scores = xgb.cv(dtrain=DX_plus, params=params, nfold=5, num_boost_round=1000, early_stopping_rounds=5,verbose_eval=None,metrics="rmse", seed=0,show_stdv=False)


tr= pd.to_numeric(scores['train-rmse-mean'].tail(1))
ts= pd.to_numeric(scores['test-rmse-mean'].tail(1))
print("Average Cross Validation RMSE score (across experiments):")
print(tr)
print(ts)


Average Cross Validation RMSE score (across experiments):
180    0.08809
Name: train-rmse-mean, dtype: float64
180    0.128595
Name: test-rmse-mean, dtype: float64


In [10]:
my_pipeline.fit(X_plus,y)

train_preds = my_pipeline.predict(X_plus)
train_mse = sqrt(np.square(train_preds-y).mean(axis=None))
print("Training RMSE score :")
print(train_mse)


/opt/conda/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


Training RMSE score :
0.08181133925079583


In [11]:
test_plus = test.copy()

for col in cols_with_missing:
    test_plus[col + '_was_missing'] = test_plus[col].isnull()

preds = my_pipeline.predict(test_plus)

submission.iloc[:,1] =np.expm1(preds)

# Write to csv
submission.to_csv("submission_2020.csv", index=False)
print('Done')


Done
